In [1]:
from nltk import sent_tokenize
import nltk
import string
from decimal import Decimal
from sklearn.model_selection import train_test_split
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wasif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#These are the things we want to have removed.
transl_table = dict( [ (ord(x), ord(y)) for x,y in zip( u"‘’´“”–-",  u"'''\"\"--") ] ) 

filename = 'doyle_Bohemia.txt'
file = open(filename, encoding="utf8")
text = file.read().replace('\n', ' ')
file.close()

In [5]:
#Clean up our data file.
sentences = sent_tokenize(text)
newSentence = [];

tempSentence = ""
for sentence in sentences:
    tempSentence = ""
    temp = sentence.translate(transl_table)
    for word in temp.split():
        word = word.replace('"', '')
        word = word.replace(',', '')
        word = word.replace('.', '')
        word = word.replace('\'', '')
        word = word.replace('?', '')
        word = word.replace('-', '')
        word = word.replace('_', '')
        word = word.lower()
        tempSentence += word + " "
    newSentence.append(tempSentence)
text_file = open("doyle_Bohemia_clean.txt", "w")

for sent in newSentence:
    n = text_file.write(sent + "\n")
    
text_file.close()

In [6]:
#Part 1 & Part 2

#Unique words in our datafile
dictionary = {}

#total words in Train set
total_words = 0

#Total words in test set
test_words = 0

#We have 449 lines, which roughly translates to 359 i.e. 80%
train, test = train_test_split(newSentence, test_size = 0.2, random_state = 42)

#Find the number of occurances of each word
for sentence in train:
    for word in sentence.split():
        total_words += 1
        word = word.lower()
        if word not in dictionary:
            dictionary[word] = 1
        else:
            dictionary[word] = dictionary[word] + 1

for sentence in test:
    for word in sentence.split():
        test_words += 1
                    

In [7]:
f = open("unigram_probs.txt", "w")
for entry in dictionary:
    f.write(entry + ": " + str(dictionary[entry]/total_words) + "\n")
    
f.close()

In [8]:
#Part 3
bigram_dict = {}

#Create counts of bigrams
for sentence in train:
    word = sentence.split()
    for i in range(len(word) - 1):
        word[i] = word[i].lower()
        if word[i] not in bigram_dict:
            bigram_dict[word[i]] = {}
            bigram_dict[word[i]][word[i+1]] = 1
        else: 
            if word[i+1] not in bigram_dict[word[i]]:
                bigram_dict[word[i]][word[i+1]] = 1
            else:
                bigram_dict[word[i]][word[i+1]] = bigram_dict[word[i]][word[i+1]] + 1
                

f = open("bigram_probs.txt", "w")
for entry in bigram_dict:
    for subentry in bigram_dict[entry]:
        f.write(entry + " " + subentry + ": " + str(bigram_dict[entry][subentry]/dictionary[entry]) + "\n")
    
f.close()



In [9]:
#Part 4 Add Smoothing
f = open("smooth_probs.txt", "w")
for entry in bigram_dict:
    for subentry in bigram_dict[entry]:
        f.write(entry + " " + subentry + ": " + str((bigram_dict[entry][subentry]+0.1)/(dictionary[entry] + 0.1 * len(dictionary))) + "\n")
    
f.close()

In [10]:
#Part 5

#We will need sentence probabilities under each model for part 6 so we collect those values now.
uni_sent = []
bi_sent = []
smoothbi_sent = []

#We calculate the probability of each sentence in the Unigram, Bigram and Smoothened Bigram model.
sentprob = 1
f = open("unigram_eval.txt", "w")
for sentence in test:
    for word in sentence:
        if word.lower() not in dictionary:
            sentprob *= 0
        else:
            sentprob *= Decimal(dictionary[word]/test_words);
    f.write(sentence + " (PROBABILITY): " + str(sentprob) +"\n")
    uni_sent.append(sentprob)
    sentprob = 1


bigramprob = 1
f = open("bigram_eval.txt", "w")
for sentence in test:
    words = sentence.split()
    for i in range(len(words) - 1):
        if words[i].lower() in bigram_dict:
            if words[i+1].lower() in bigram_dict[words[i].lower()]:
                bigramprob *= Decimal(bigram_dict[words[i]][words[i+1]]/dictionary[words[i]])
            else:
                bigramprob *= 0
        else:
            bigramprob *= 0
    f.write(sentence + " (PROBABILITY): " + str(bigramprob) +"\n")
    bi_sent.append(bigramprob)
    bigramprob = 1


smoothprob = 1
f = open("smoothed_eval.txt", "w")
for sentence in test:
    words = sentence.split()
    for i in range(len(words) - 1):
        if words[i].lower() in bigram_dict:
            if words[i+1].lower() in bigram_dict[words[i].lower()]:
                smoothprob *= (bigram_dict[words[i].lower()][words[i+1].lower()]+0.1)/(dictionary[words[i].lower()] + 0.1 * len(dictionary))
            else:
                smoothprob *= (0+0.1)/(dictionary[entry] + 0.1 * len(dictionary))
        else:
            smoothprob *= (0.1)/(0.1*len(dictionary))
    f.write(sentence + " (PROBABILITY): " + str(smoothprob) +"\n")
    smoothbi_sent.append(smoothprob)
    smoothprob = 1



In [13]:
#Now we calculate the perplexity of each Model.
f = open("Readme.txt", "w")
f.write("Name: Ali Iftakhar \n \n")
uni_perp = 1

#Calculate P(s_1, s_2, .... s_n)
for i in range(len(uni_sent)):
    uni_perp *= Decimal(uni_sent[i])
        
bi_perp = 1
#Calculate P(s_1, s_2, .... s_n)
for i in range(len(bi_sent)):
    bi_perp *= Decimal(bi_sent[i])
    
smooth_bi_perp = 1.0
#Calculate P(s_1, s_2, .... s_n)
for i in range(len(smoothbi_sent)):
    #This was just an attempt to stomp python's rounding to 0 since the number is really small.
    if(smooth_bi_perp * smoothbi_sent[i] != 0):
        smooth_bi_perp *= smoothbi_sent[i]


if uni_perp == 0:
    f.write("Perplexity of Unigram: Infinity \n")
else:
    final_uni = (1/uni_perp)**(len(test))
    f.write("Perplexity of Unigram: " + final_uni)
    

if bi_perp == 0:
    f.write("Perplexity of Bigram: Infinity \n")
else:
    final_bi = (1/bi_perp)**(len(test))
    f.write("Perplexity of Bigram: " + final_bi)
    
#It seems like since smooth_bi_prep is such a miniscule value, it ends up defaulting to infinity, but I am confident that 
#there exists an answer, but it is just way too big.
final_smooth_bi = (1.0/smooth_bi_perp)**(len(test))
f.write("Perplexity of Smoothened Bigram: " + str(final_smooth_bi) + "\n")


f.write("ANSWER TO QUESTION 1 \n")
f.write("I think the worst performance by far, came from the Unigram model. Since there is no dependency of the word to the prior word, we cannot hope to achieve a good prediction. It is unfortunate though that I wasn't aware to get number values for perplexity \n \n")

f.write("ANSWER TO QUESTION 2 \n")
f.write("I think smoothing definitely helped with the overall performance. For unigram and bigram when we had a 0 probability sentence, it ruined the entire perplexity calculation. Sadly, my probabilities even with smoothing were so low that I wasn't able to get a finite number answer for Smoothened Bigram as well. I'm not sure what I did wrong.")
f.close()